This notebook performs profile annotation.
The platemap is mapped back to the profile to retain the sample metadata.


In [1]:
import argparse
import os
import pathlib
import sys

import pandas as pd
from arg_parsing_utils import parse_args
from notebook_init_utils import bandicoot_check, init_notebook

root_dir, in_notebook = init_notebook()

profile_base_dir = bandicoot_check(
    pathlib.Path(os.path.expanduser("~/mnt/bandicoot/NF1_organoid_data")).resolve(),
    root_dir,
)

In [2]:
if not in_notebook:
    args = parse_args()
    patient = args["patient"]
    image_based_profiles_subparent_name = args["image_based_profiles_subparent_name"]

else:
    patient = "NF0014_T1"
    image_based_profiles_subparent_name = "image_based_profiles"

In [3]:
def annotate_profiles(
    profile_df: pd.DataFrame, platemap_df: pd.DataFrame, patient: str
) -> pd.DataFrame:
    """
    Annotate profiles with treatment, dose, and unit information from the platemap.

        Parameters
        ----------
        profile_df : pd.DataFrame
            Profile DataFrame containing image_set information.
            Could be either single-cell or organoid profiles.
        platemap_df : pd.DataFrame
            Platmap DataFrame containing well_position, treatment, dose, and unit.
        patient : str
            Patient ID to annotate the profiles with.

        Returns
        -------
        pd.DataFrame
            Annotated profile DataFrame with additional columns for treatment, dose, and unit.
    """
    drug_information = pd.read_csv(
        pathlib.Path(
            f"{root_dir}/4.processing_image_based_profiles/data/drugs/drug_information.csv"
        )
    )
    profile_df["Well"] = profile_df["image_set"].str.split("-").str[0]
    profile_df.insert(2, "Well", profile_df.pop("Well"))
    profile_df = pd.merge(
        profile_df,
        platemap_df[["well_position", "treatment", "dose", "unit"]],
        left_on="Well",
        right_on="well_position",
        how="left",
    ).drop(columns=["well_position"])
    profile_df = profile_df.merge(
        drug_information, how="left", left_on="treatment", right_on="Treatment"
    )
    profile_df.drop(columns=["Treatment"], inplace=True)
    for col in ["treatment", "dose", "unit"]:
        profile_df.insert(1, col, profile_df.pop(col))
    profile_df.insert(0, "patient", patient)
    return profile_df

## pathing 

In [4]:
sc_merged_path = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/{image_based_profiles_subparent_name}/1.combined_profiles/sc.parquet"
).resolve(strict=True)
organoid_merged_path = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/{image_based_profiles_subparent_name}/1.combined_profiles/organoid.parquet"
).resolve(strict=True)

platemap_path = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/platemap/platemap.csv"
).resolve(strict=True)

# output path
sc_annotated_output_path = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/{image_based_profiles_subparent_name}/2.annotated_profiles/sc_anno.parquet"
).resolve()
organoid_annotated_output_path = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/{image_based_profiles_subparent_name}/2.annotated_profiles/organoid_anno.parquet"
).resolve()

organoid_annotated_output_path.parent.mkdir(parents=True, exist_ok=True)

In [5]:
# read data
sc_merged = pd.read_parquet(sc_merged_path)
organoid_merged = pd.read_parquet(organoid_merged_path)
# read platemap
platemap = pd.read_csv(platemap_path)
platemap.head()

,WellRow,WellCol,well_position,treatment,dose,unit
0,C,2,C2,Staurosporine,10,nM
1,D,2,D2,Digoxin,1,uM
2,E,2,E2,Digoxin,1,uM
3,F,2,F2,Onalespib,1,uM
4,G,2,G2,Staurosporine,10,nM


In [6]:
sc_merged = annotate_profiles(sc_merged, platemap, patient)
organoid_merged = annotate_profiles(organoid_merged, platemap, patient)

In [7]:
sc_merged.rename(columns={"patient": "patient_tumor"}, inplace=True)
organoid_merged.rename(columns={"patient": "patient_tumor"}, inplace=True)
sc_merged[["patient", "tumor"]] = sc_merged["patient_tumor"].str.split("_", expand=True)
organoid_merged[["patient", "tumor"]] = organoid_merged["patient_tumor"].str.split(
    "_", expand=True
)

In [8]:
metadata_features_list = [
    "patient_tumor",
    "patient",
    "tumor",
    "object_id",
    "unit",
    "dose",
    "Well",
    "treatment",
    "image_set",
    "parent_organoid",
    "single_cell_count",
    "Target",
    "Class",
    "Therapeutic_Categories",
]
# prepend "Metadata_" to metadata features
sc_merged = sc_merged.rename(
    columns={col: f"Metadata_{col}" for col in metadata_features_list}
)
organoid_merged = organoid_merged.rename(
    columns={col: f"Metadata_{col}" for col in metadata_features_list}
)

In [9]:
# save annotated profiles
sc_merged.to_parquet(sc_annotated_output_path, index=False)
organoid_merged.to_parquet(organoid_annotated_output_path, index=False)

In [10]:
organoid_merged.head()

,Metadata_patient_tumor,Metadata_object_id,Metadata_unit,Metadata_dose,Metadata_treatment,Metadata_image_set,Metadata_Well,Metadata_single_cell_count,Area.Size.Shape_Organoid_VOLUME,Area.Size.Shape_Organoid_CENTER.X,...,Texture_Organoid_Mito_Inverse.Difference.Moment_256.3,Texture_Organoid_Mito_Sum.Average_256.3,Texture_Organoid_Mito_Sum.Entropy_256.3,Texture_Organoid_Mito_Sum.Variance_256.3,Texture_Organoid_Mito_Variance_256.3,Metadata_Target,Metadata_Class,Metadata_Therapeutic_Categories,Metadata_patient,Metadata_tumor
0,NF0014_T1,2,uM,1,Fimepinostat,E5-2,E5,NaN,28.0,1440.178589,...,0.999998,0.000295,0.000038,0.052777,0.026389,PI3K and HDAC inhibitor,Small Molecule,Investigational,NF0014,T1
1,NF0014_T1,4,uM,1,Fimepinostat,E5-2,E5,NaN,3931.0,1433.475952,...,0.999795,0.027184,0.003727,3.917699,1.393737,PI3K and HDAC inhibitor,Small Molecule,Investigational,NF0014,T1
2,NF0014_T1,7,uM,1,Fimepinostat,E5-2,E5,NaN,86.0,1025.965088,...,0.999992,0.001290,0.000172,0.204001,0.102001,PI3K and HDAC inhibitor,Small Molecule,Investigational,NF0014,T1
3,NF0014_T1,19,uM,1,Fimepinostat,E5-2,E5,NaN,532.0,1192.227417,...,0.999957,0.005629,0.000805,0.754543,0.366985,PI3K and HDAC inhibitor,Small Molecule,Investigational,NF0014,T1
4,NF0014_T1,23,uM,1,Fimepinostat,E5-2,E5,NaN,1907836.0,1272.132568,...,0.927072,8.438087,0.831688,834.310213,227.063283,PI3K and HDAC inhibitor,Small Molecule,Investigational,NF0014,T1


In [11]:
sc_merged.head()

,Metadata_patient_tumor,Metadata_object_id,Metadata_unit,Metadata_dose,Metadata_treatment,Metadata_image_set,Metadata_Well,Metadata_parent_organoid,Area.Size.Shape_Nuclei_VOLUME,Area.Size.Shape_Nuclei_CENTER.X,...,Texture_Cytoplasm_Mito_Inverse.Difference.Moment_256.3,Texture_Cytoplasm_Mito_Sum.Average_256.3,Texture_Cytoplasm_Mito_Sum.Entropy_256.3,Texture_Cytoplasm_Mito_Sum.Variance_256.3,Texture_Cytoplasm_Mito_Variance_256.3,Metadata_Target,Metadata_Class,Metadata_Therapeutic_Categories,Metadata_patient,Metadata_tumor
0,NF0014_T1,61,uM,1,Fimepinostat,E5-2,E5,-1,33670.0,407.706512,...,0.996985,0.444618,0.048899,72.962977,21.955133,PI3K and HDAC inhibitor,Small Molecule,Investigational,NF0014,T1
1,NF0014_T1,114,uM,1,Fimepinostat,E5-2,E5,-1,55176.0,570.549927,...,0.993073,0.904632,0.104650,133.006588,40.191502,PI3K and HDAC inhibitor,Small Molecule,Investigational,NF0014,T1
2,NF0014_T1,184,uM,1,Fimepinostat,E5-2,E5,-1,91660.0,1244.865967,...,0.991571,0.710197,0.119023,62.319702,19.047745,PI3K and HDAC inhibitor,Small Molecule,Investigational,NF0014,T1
3,NF0014_T1,219,uM,1,Fimepinostat,E5-2,E5,-1,6752.0,654.586487,...,0.997428,0.275313,0.043949,31.727164,9.307586,PI3K and HDAC inhibitor,Small Molecule,Investigational,NF0014,T1
4,NF0014_T1,255,uM,1,Fimepinostat,E5-2,E5,-1,14346.0,443.827972,...,0.997048,0.508077,0.047548,93.847486,28.481513,PI3K and HDAC inhibitor,Small Molecule,Investigational,NF0014,T1
